In [1]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Load forecasted sales data
forecasted_sales = pd.read_csv('./outputs/forecasted_sales.csv')

# Validate that data is loaded correctly
print(forecasted_sales.head())

# Define inventory parameters
safety_stock = 50  # Safety stock level
reorder_point = 100  # Reorder threshold
max_inventory_per_day = 100  # Maximum inventory per day
cost_per_unit_inventory = 5  # Cost per unit of inventory

# Calculate basic optimal inventory levels
forecasted_sales['optimal_inventory'] = np.maximum(
    forecasted_sales['predicted_sales'] + safety_stock - reorder_point, 0
)

# Linear Programming Model for Inventory Optimization
model = LpProblem("Inventory_Optimization", LpMinimize)

# Create decision variables for inventory ordering
inventory_vars = [
    LpVariable(f"inventory_{i}", lowBound=0, cat="Integer")
    for i in range(len(forecasted_sales))
]

# Objective: Minimize inventory holding cost
model += lpSum([inventory_vars[i] * cost_per_unit_inventory for i in range(len(inventory_vars))])

# Constraints
for i, row in forecasted_sales.iterrows():
    # Inventory must be sufficient to cover predicted sales
    model += inventory_vars[i] + row['optimal_inventory'] >= row['predicted_sales'], f"Stockout_{i}"
    # Inventory ordered must not exceed the maximum inventory per day
    model += inventory_vars[i] <= max_inventory_per_day, f"MaxInventory_{i}"

# Solve the optimization problem
model.solve()

# Update DataFrame with optimized inventory levels
if model.status == 1:  # Check if the solution is optimal
    forecasted_sales['optimized_inventory'] = [inventory_var.varValue for inventory_var in inventory_vars]
    print("Optimization successful. Updated inventory levels:")
    print(forecasted_sales[['date', 'predicted_sales', 'optimized_inventory']])
else:
    print("No optimal solution found.")

# Save optimized inventory levels to CSV
output_path = './outputs/optimized_inventory.csv'
forecasted_sales[['date', 'predicted_sales', 'optimized_inventory']].to_csv(output_path, index=False)
print(f"Optimized inventory levels saved to '{output_path}'")


   date  predicted_sales
0   0.0         0.810927
Optimization successful. Updated inventory levels:
   date  predicted_sales  optimized_inventory
0   0.0         0.810927                  1.0
Optimized inventory levels saved to './outputs/optimized_inventory.csv'
